In [1]:
#Importing necessary libraries and pyomo objects
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np

In [2]:
#Creating the model object
RGenModel = AbstractModel()

#Sets & Parameters of the Abstract model
RGenModel.gen = Set()                     #non-renewable generators
RGenModel.t = Set()                       #time periods 
RGenModel.rgen = Set()                    #renewable generators

#Operational cost of each generator --> heat_rate*fuel_cost + VOM + FOM
RGenModel.OpCost = Param(RGenModel.gen)
RGenModel.ROpCost = Param(RGenModel.rgen)
#Capacity of each generator 
RGenModel.Cap = Param(RGenModel.gen)
RGenModel.RCap = Param(RGenModel.rgen)
#Demand at any period of time --> hourly
RGenModel.Demand = Param(RGenModel.t, within=PositiveReals)
#Capacity factor for renewable generators at a given period of time
RGenModel.CF = Param(RGenModel.rgen, RGenModel.t)

#Declaring decision variable
#Electricity generation for any generator at a given period of time
RGenModel.EG = Var(RGenModel.gen, RGenModel.t, domain=NonNegativeReals)
RGenModel.REG = Var(RGenModel.rgen, RGenModel.t, domain=NonNegativeReals)

In [3]:
#Objective function
#Minimize the operational system cost over the fleet of non-renewable and renewable generators

def obj_expression(RGenModel):
    return sum(sum(RGenModel.OpCost[g]*RGenModel.EG[g,t] for g in RGenModel.gen) for t in RGenModel.t) + \
           sum(sum(RGenModel.ROpCost[rg]*RGenModel.REG[rg,t] for rg in RGenModel.rgen) for t in RGenModel.t)

RGenModel.sysCost = Objective(rule=obj_expression, sense=minimize)

In [4]:
#The above objective function is subject to multiple constraints

#1. Supply == Demand for all periods of time
def balance_rule(RGenModel, t):
    return sum(RGenModel.EG[g,t] for g in RGenModel.gen) + \
           sum(RGenModel.REG[rg,t] for rg in RGenModel.rgen) == RGenModel.Demand[t]

RGenModel.balance = Constraint(RGenModel.t, rule=balance_rule)

In [5]:
#2. The Generation potential of each non-renewable generator is restricted by its max capacity
def max_gen_rule(RGenModel, gen, t):
    return RGenModel.EG[gen,t] <= RGenModel.Cap[gen]

RGenModel.max_gen = Constraint(RGenModel.gen, RGenModel.t, rule=max_gen_rule)

In [6]:
#3. The generation potential of renewable generators are limited by its capacity factor
def max_gen_renewable_rule(RGenModel, rgen, t):
    return RGenModel.REG[rgen,t] <= RGenModel.RCap[rgen] * CF_dict[rgen,t]

RGenModel.max_rgen = Constraint(RGenModel.rgen, RGenModel.t, rule=max_gen_renewable_rule)

In [50]:
#Specifying the solver to use for optimization along with Dataportal
opt = SolverFactory('gurobi')
data = DataPortal()

#Reading respective demand, generator and capacity factor profiles from different 
data.load(filename='Hourly_demand_profile_2021.csv', format='set', set='t', model=RGenModel)
data.load(filename='Hourly_demand_profile_2021.csv', index='t', param='Demand', model=RGenModel)
data.load(filename='generation_data.csv', format='set', set='gen', model=RGenModel)
data.load(filename='renewable_generation_data.csv', format='set', set='rgen', model=RGenModel)
data.load(filename='generation_data.csv',index='gen', param=['OpCost','Cap'], model=RGenModel)
data.load(filename='renewable_generation_data.csv', index='rgen',param=['ROpCost','RCap'], model=RGenModel)
#data.load(filename='Renewable_generators_CF.csv', index=['rgen','t'], param='CF', model=RGenModel)
#Reading the CF file and post processing using pandas to handle multi-index 
CF_dict = pd.read_csv('Renewable_generators_CF.csv', index_col='t').unstack(0).to_dict()

#Creating an instance and displaying all information of that instance
instance = RGenModel.create_instance(data)

#We can display all the info of the instance
#instance.pprint()

#Solving the optimization problem
results = opt.solve(instance, symbolic_solver_labels=True, tee=True) 

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-09
Read LP format model from file /var/folders/rk/rn44mwf56119gm2tgnf1dcjw0000gn/T/tmpnx47nk7c.pyomo.lp
Reading time = 0.25 seconds
sysCost: 87601 rows, 78841 columns, 157681 nonzeros
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 87601 rows, 78841 columns and 157681 nonzeros
Model fingerprint: 0x72fb5e3f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 7e+04]
Presolve removed 84432 rows and 57467 columns
Presolve time: 0.18s
Presolved: 3169 rows, 21374 columns, 21374 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ 